In [1]:
import torch
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel

/Users/lcjh/Library/Python/3.9/lib/python/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/Users/lcjh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
try:
    import accelerate
    print(f"accelerate is installed: version {accelerate.__version__}")
except ImportError:
    print("accelerate is NOT installed in this environment")

accelerate is installed: version 1.10.1


In [3]:
# get the device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

In [4]:
!ls

blog_pics
CLIP_Embedding_samples_cats_pineapples.ipynb
images
README.md


In [6]:
# 1. Load pretrained CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [7]:
# 2. Load example images
cat_image = Image.open("images/cat.png")
pineapple_image = Image.open("images/pineapple.png")

In [10]:
# 3. Define the text prompts
texts = ["a photo of a cat", "a photo of a pineapple"]

# 4. Preprocess the data
inputs = processor(
    text=texts,
    images=[cat_image, pineapple_image],
    return_tensors="pt",
    padding=True
).to(device)

In [11]:
# 5. Forward pass
with torch.no_grad():
    outputs = model(**inputs)
    image_embeds = outputs.image_embeds    # shape [2, 512]
    text_embeds = outputs.text_embeds      # shape [2, 512]

In [12]:
# 6. Normalize embeddings (L2 normalization)
image_embeds = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)

In [13]:
# 7. Compute cosine similarities
cosine_sim = image_embeds @ text_embeds.T  # 2x2 matrix
print("Cosine similarity matrix:")
print(cosine_sim)

Cosine similarity matrix:
tensor([[0.2832, 0.2036],
        [0.1956, 0.3286]], device='mps:0')
